In [1]:
import pyodbc
import pandas as pd
import numpy as np
from datetime import datetime
%matplotlib inline

c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\users\jhpark1\anaconda3\envs\tc\lib\site-packages\numpy\.libs\libopenblas.TXA6YQSD3GCQQC22GEQ54J2UDCXDXHWN.gfortran-win_amd64.dll
  stacklevel=1)


In [2]:
conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                     'Server=🧡;'
                     'Database=💛;'
                     'UID=💚;'
                     'PWD=💜')

In [3]:
cursor = conn.cursor()

In [4]:
# 개별시세 불러오기 (전체)
# df1 = pd.read_sql_query("SELECT * FROM (SELECT D.아파트코드, D.아파트명, D.입주연도,D.도시, D.구시군 , D.읍면동, C.평형, C.평형타입,C.세대수, C.조사일, C.매매하한가, C.매매상한가 FROM (SELECT A.아파트코드, A.평형, A.평형타입, A.세대수, B.조사일, B.매매하한가, B.매매상한가 FROM modu.P_APTSUB AS A INNER JOIN modu.P_APTHPRICE AS B ON A.아파트코드 = B.아파트코드 AND A.평형 = B.평형 AND A.평형타입 = B. 평형타입 WHERE B.조사일 > 19970000 and B.조사일 < 20140000) AS C INNER JOIN modu.P_APT AS D ON C.아파트코드 = D.아파트코드) AS E WHERE E.도시 = '서울특별시' and E.구시군 = '강남구'", conn)
df = pd.read_sql_query("SELECT * FROM (SELECT D.아파트코드, D.아파트명, D.입주연도,D.도시, D.구시군 , D.읍면동, C.평형, C.평형타입,C.세대수, C.주택형, C.분양가, C.조사일, C.매매하한가, C.매매상한가, C.전세하한가, C.전세상한가 FROM (SELECT A.아파트코드, A.평형, A.평형타입, A.세대수, B.조사일, A.주택형, A.분양가, B.매매하한가, B.매매상한가, B.전세하한가, B.전세상한가 FROM modu.P_APTSUB AS A INNER JOIN modu.P_APTHPRICE AS B ON A.아파트코드 = B.아파트코드 AND A.평형 = B.평형 AND A.평형타입 = B. 평형타입) AS C INNER JOIN modu.P_APT AS D ON C.아파트코드 = D.아파트코드) AS E WHERE E.매매하한가 > 0 AND E.조사일 NOT LIKE '% %'", conn)

# SQL 쿼리의 조회조건으로 대체
# df = df[df["aptprice"] != 0]
# df = df[(df["조사일"].str.contains(' ')) == False]

df["aptprice"] = df[['매매하한가','매매상한가']].mean(axis=1)
df["aptprice_all_sum"] = df["aptprice"] * df["세대수"]
df["js_price"] = df[['전세하한가', '전세상한가']].mean(axis=1)
df["js_rate"] = (df["js_price"] / df["aptprice"]) * 100
# df["price_sq"] = df["aptprice"] / df["주택형"]
# df["price_py"] = df["price_sq"] * 3.305785
df["조사일"] = df["조사일"].astype(int)
df = df[-df["입주연도"].astype(str).str.startswith('0')]
df.head()

,아파트코드,아파트명,입주연도,도시,구시군,읍면동,평형,평형타입,세대수,주택형,분양가,조사일,매매하한가,매매상한가,전세하한가,전세상한가,aptprice,aptprice_all_sum,js_price,js_rate
0,A01141201100012,삼성빌라,1987,서울특별시,서대문구,연희동,00028,,48,92.5624,0,20200612,28000,30000,18000,18000,29000.0,1392000.0,18000.0,62.068966
1,A01141201100025,일진아이윌,20050530,서울특별시,서대문구,연희동,00029,,3,97.9500,26820,20200612,50000,55000,28000,30000,52500.0,157500.0,29000.0,55.238095
2,A01141201200004,백련미라보,199505,서울특별시,서대문구,남가좌동,00016,,2,48.3800,0,20200612,12000,14000,6000,7000,13000.0,26000.0,6500.0,50.000000
3,A01141201300014,일신휴먼빌,20060215,서울특별시,서대문구,북가좌동,00031,N,52,103.8100,27800,20200612,59000,63000,39000,42000,61000.0,3172000.0,40500.0,66.393443
4,A01141201300016,신일해피트리,20051029,서울특별시,서대문구,북가좌동,00033,B,44,109.1300,27654,20200612,59000,64000,36000,40000,61500.0,2706000.0,38000.0,61.788618


In [5]:
import datetime
import timedelta
# 오늘날짜 기준
dt = datetime.datetime.now()
fl_dt = str(dt.year) + str(dt.month) + str(dt.day)

In [6]:
# 전국 아파트 시가총액 집계
dat2 = df.groupby(["조사일"])["aptprice_all_sum"].sum()
dat2 = dat2.reset_index()
dat2["지역명"] = "전국"
dat2 = dat2[["조사일","지역명","aptprice_all_sum"]]
dat2.to_csv('all_apt_sum_' + fl_dt + '.csv')
dat2

,조사일,지역명,aptprice_all_sum
0,19970115,전국,1.285458e+10
1,19970215,전국,1.301018e+10
2,19970315,전국,1.313447e+10
3,19970415,전국,1.322415e+10
4,19970515,전국,1.337939e+10
...,...,...,...
1103,20200515,전국,3.285549e+11
1104,20200522,전국,3.288353e+11
1105,20200529,전국,3.289611e+11
1106,20200605,전국,3.291082e+11


In [7]:
# 구시군별 아파트 시가총액 집계
dat3 = df.groupby(["조사일","도시"])["aptprice_all_sum"].sum()
dat3 = dat3.reset_index()
dat3 = dat3.rename(columns={'도시' : '지역명'})
dat3.to_csv('city_apt_sum_' + fl_dt + '.csv')
dat3

,조사일,지역명,aptprice_all_sum
0,19970115,강원도,1.109630e+07
1,19970115,경기도,4.355802e+09
2,19970115,경상남도,1.816817e+08
3,19970115,경상북도,4.546000e+06
4,19970115,광주광역시,1.086240e+08
...,...,...,...
18633,20200612,전라남도,1.890538e+09
18634,20200612,전라북도,3.727527e+09
18635,20200612,제주도,9.296834e+08
18636,20200612,충청남도,5.977351e+09


In [8]:
# 구시군별 아파트 시가총액 집계
dat4 = df[df["도시"] == "서울특별시"].groupby(["조사일","구시군"])["aptprice_all_sum"].sum()
dat4 = dat4.reset_index()
dat4 = dat4.rename(columns={'구시군' : '지역명'})
dat4.to_csv('sgg_apt_sum_' + fl_dt + '.csv')
dat4

,조사일,지역명,aptprice_all_sum
0,19970115,강남구,1.164137e+09
1,19970115,강동구,2.478500e+08
2,19970115,강북구,5.459925e+07
3,19970115,강서구,2.640251e+08
4,19970115,관악구,1.088115e+08
...,...,...,...
27695,20200612,용산구,3.834151e+09
27696,20200612,은평구,2.499413e+09
27697,20200612,종로구,9.896810e+08
27698,20200612,중구,1.389512e+09


In [9]:
# 구시군별 아파트 시가총액 집계
dat5 = df[df["도시"] == "경기도"].groupby(["조사일","구시군"])["aptprice_all_sum"].sum()
dat5 = dat5.reset_index()
dat5 = dat5.rename(columns={'구시군' : '지역명'})
dat5.to_csv('gg_apt_sum_' + fl_dt + '.csv')
dat5

,조사일,지역명,aptprice_all_sum
0,19970115,고양시,7.953738e+08
1,19970115,과천시,1.063462e+08
2,19970115,광명시,3.314012e+08
3,19970115,구리시,9.237360e+07
4,19970115,군포시,2.631987e+08
...,...,...,...
33309,20200612,파주시,2.043920e+09
33310,20200612,평택시,2.124989e+09
33311,20200612,포천시,1.834887e+08
33312,20200612,하남시,2.823756e+09


In [10]:
dat_all = pd.concat([dat2, dat3],axis = 0)
dat_all = pd.concat([dat_all,dat4], axis = 0)
dat_all = pd.concat([dat_all,dat5], axis = 0)
dat_all.to_csv('dat_all_' + fl_dt + '.csv')
dat_all

,조사일,지역명,aptprice_all_sum
0,19970115,전국,1.285458e+10
1,19970215,전국,1.301018e+10
2,19970315,전국,1.313447e+10
3,19970415,전국,1.322415e+10
4,19970515,전국,1.337939e+10
...,...,...,...
33309,20200612,파주시,2.043920e+09
33310,20200612,평택시,2.124989e+09
33311,20200612,포천시,1.834887e+08
33312,20200612,하남시,2.823756e+09
